### 해야할 것
- 입출구코드별로 톨게이트 이름 넣어서 시각화에 hover_name 지정

In [251]:
%matplotlib inline
import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [70]:
# matplotlib에 “AppleGothc”(Mac) 또는 “Malgun Gothic”(Windows) 글꼴 적용

import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

## 봐야할 톨게이트 코드 -> 총 10개

김포 68 - 김포

인천 11 - 인천

서서울 253 - 안산

청계 64 - 의왕

판교 65 - 성남

대왕판교 69 - 성남

성남 61 - 성남

동서울 190 - 하남 

하남 66 - 하남

구리남양주 62 - 구리

### 2016년 데이터 정제 (서울꺼만)

In [211]:
data_list_2016 = ['1602.csv', '1603.csv', '1604.csv', '1605.csv', '1606.csv',
            '1607.csv', '1608.csv', '1609.csv', '1610.csv', '1611.csv', '1612.csv']

In [212]:
data_2016_1 = pd.read_csv('1601.csv', encoding = 'euc_kr')
data_2016_1.head()

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20160101,11,0,1,0,1,14882,172,173,164,139,1556,17086
1,20160101,11,0,2,0,1,33301,421,385,105,93,1981,36286
2,20160101,11,1,1,0,1,15352,113,180,162,112,1714,17633
3,20160101,11,1,2,0,1,35129,459,437,119,99,2303,38546
4,20160101,12,0,1,0,1,477,2,7,3,2,32,523


In [213]:
data_2016 = data_2016_1.copy()

In [214]:
data_2016.columns

Index(['집계일자', '영업소코드', '입출구구분코드', 'TCS하이패스구분코드', '고속도로운영기관구분코드', '영업형태구분코드',
       '1종교통량', '2종교통량', '3종교통량', '4종교통량', '5종교통량', '6종교통량', '총교통량'],
      dtype='object')

In [215]:
data_2016 = data_2016[data_2016['영업소코드'] == 11]
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 68]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 253]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 64]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 65]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 69]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 61]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 190]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 66]])
data_2016 = pd.concat([data_2016, data_2016_1[data_2016_1['영업소코드'] == 62]])

for csvfile in data_list_2016:
    data_tmp = pd.read_csv(csvfile, encoding='euc_kr')
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 11]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 68]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 253]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 64]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 65]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 69]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 61]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 190]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 66]])
    data_2016 = pd.concat([data_2016, data_tmp[data_tmp['영업소코드'] == 62]])

In [216]:
data_2016

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20160101,11,0,1,0,1,14882,172,173,164,139,1556,17086
1,20160101,11,0,2,0,1,33301,421,385,105,93,1981,36286
2,20160101,11,1,1,0,1,15352,113,180,162,112,1714,17633
3,20160101,11,1,2,0,1,35129,459,437,119,99,2303,38546
1479,20160102,11,0,1,0,1,14470,271,247,280,367,1651,17286
1480,20160102,11,0,2,0,1,36147,640,440,141,350,2315,40033
1481,20160102,11,1,1,0,1,14983,232,314,254,381,1700,17864
1482,20160102,11,1,2,0,1,37881,712,459,148,388,2461,42049
2954,20160103,11,0,1,0,1,12534,168,140,136,181,1401,14560
2955,20160103,11,0,2,0,1,30142,293,390,74,154,2039,33092


In [217]:
data_2016['영업소코드'].unique()

array([ 11,  68, 253,  64,  65,  69,  61, 190,  66,  62], dtype=int64)

### 2017년 데이터 정제

In [218]:
data_list_2017 = ['1702.csv', '1703.csv', '1704.csv', '1705.csv', '1706.csv',
            '1707.csv', '1708.csv', '1709.csv', '1710.csv', '1711.csv', '1712.csv']

In [219]:
data_2017_1 = pd.read_csv('1701.csv', encoding = 'euc_kr')
data_2017_1.head()

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20170102,11,0,1,0,1,12045,561,472,577,894,1759,16308
1,20170102,11,0,2,0,1,48438,1663,779,585,1738,3675,56878
2,20170102,11,1,1,0,1,13285,332,719,478,783,1864,17461
3,20170102,11,1,2,0,1,48655,1569,787,568,1940,3768,57287
4,20170102,12,0,1,0,1,274,11,11,10,3,20,329


In [220]:
data_2017_1.columns

Index(['집계일자', '영업소코드', '입출구구분코드', 'TCS하이패스구분코드', '고속도로운영기관구분코드', '영업형태구분코드',
       '1종교통량', '2종교통량', '3종교통량', '4종교통량', '5종교통량', '6종교통량', '총교통량'],
      dtype='object')

In [221]:
data_2017 = data_2017_1.copy()
data_2017 = data_2017[data_2017['영업소코드'] == 11]
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 68]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 253]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 64]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 65]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 69]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 61]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 190]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 66]])
data_2017 = pd.concat([data_2017, data_2017_1[data_2017_1['영업소코드'] == 62]])

for csvfile in data_list_2017:
    data_tmp = pd.read_csv(csvfile, encoding='euc_kr')
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 11]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 68]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 253]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 64]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 65]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 69]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 61]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 190]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 66]])
    data_2017 = pd.concat([data_2017, data_tmp[data_tmp['영업소코드'] == 62]])

In [222]:
data_2017

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20170102,11,0,1,0,1,12045,561,472,577,894,1759,16308
1,20170102,11,0,2,0,1,48438,1663,779,585,1738,3675,56878
2,20170102,11,1,1,0,1,13285,332,719,478,783,1864,17461
3,20170102,11,1,2,0,1,48655,1569,787,568,1940,3768,57287
1523,20170101,11,0,1,0,1,13107,137,113,76,62,1481,14976
1524,20170101,11,0,2,0,1,35199,283,357,111,159,2164,38273
1525,20170101,11,1,1,0,1,13902,49,129,77,46,1653,15856
1526,20170101,11,1,2,0,1,37820,268,428,118,143,2365,41142
3046,20170104,11,0,1,0,1,12892,721,524,639,1032,1907,17715
3047,20170104,11,0,2,0,1,50858,1837,858,644,2047,4006,60250


### 2018 데이터 정제

In [223]:
data_list_2018 = ['1802.csv', '1803.csv', '1804.csv', '1805.csv', '1806.csv',
            '1807.csv', '1808.csv', '1809.csv', '1810.csv', '1811.csv', '1812.csv']

In [224]:
data_2018_1 = pd.read_csv('1801.csv', encoding = 'euc_kr')
data_2018_1.head()

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20180101,11,0,1,0,1,10594,141,85,71,64,1271,12226
1,20180101,11,0,2,0,1,33747,424,381,168,181,1990,36891
2,20180101,11,1,1,0,1,11398,81,107,88,33,1354,13061
3,20180101,11,1,2,0,1,36471,396,411,164,179,2244,39865
4,20180101,12,0,1,0,1,294,1,4,2,0,29,330


In [225]:
data_2018 = data_2018_1.copy()
data_2018 = data_2018[data_2018['영업소코드'] == 11]
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 68]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 253]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 64]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 65]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 69]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 61]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 190]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 66]])
data_2018 = pd.concat([data_2018, data_2018_1[data_2018_1['영업소코드'] == 62]])

for csvfile in data_list_2018:
    data_tmp = pd.read_csv(csvfile, encoding='euc_kr')
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 11]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 68]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 253]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 64]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 65]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 69]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 61]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 190]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 66]])
    data_2018 = pd.concat([data_2018, data_tmp[data_tmp['영업소코드'] == 62]])

In [226]:
data_2018

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20180101,11,0,1,0,1,10594,141,85,71,64,1271,12226
1,20180101,11,0,2,0,1,33747,424,381,168,181,1990,36891
2,20180101,11,1,1,0,1,11398,81,107,88,33,1354,13061
3,20180101,11,1,2,0,1,36471,396,411,164,179,2244,39865
1557,20180102,11,0,1,0,1,10783,538,378,481,651,1586,14417
1558,20180102,11,0,2,0,1,48767,1577,756,599,1883,3674,57256
1559,20180102,11,1,1,0,1,11897,271,580,476,605,1685,15514
1560,20180102,11,1,2,0,1,50508,1585,824,559,2042,3730,59248
3114,20180104,11,0,1,0,1,11099,638,499,526,825,1706,15293
3115,20180104,11,0,2,0,1,50085,1667,792,637,2350,3831,59362


In [269]:
data_all = pd.concat([data_2016, data_2017])
data_all = pd.concat([data_all, data_2018])
data_all

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,고속도로운영기관구분코드,영업형태구분코드,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20160101,11,0,1,0,1,14882,172,173,164,139,1556,17086
1,20160101,11,0,2,0,1,33301,421,385,105,93,1981,36286
2,20160101,11,1,1,0,1,15352,113,180,162,112,1714,17633
3,20160101,11,1,2,0,1,35129,459,437,119,99,2303,38546
1479,20160102,11,0,1,0,1,14470,271,247,280,367,1651,17286
1480,20160102,11,0,2,0,1,36147,640,440,141,350,2315,40033
1481,20160102,11,1,1,0,1,14983,232,314,254,381,1700,17864
1482,20160102,11,1,2,0,1,37881,712,459,148,388,2461,42049
2954,20160103,11,0,1,0,1,12534,168,140,136,181,1401,14560
2955,20160103,11,0,2,0,1,30142,293,390,74,154,2039,33092


---

### 필요 없는 데이터 drop

In [270]:
data_all = data_all.drop(data_all.columns[[4, 5, 6, 7, 8, 9, 10, 11]], axis=1)
data_all.head()

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량
0,20160101,11,0,1,17086
1,20160101,11,0,2,36286
2,20160101,11,1,1,17633
3,20160101,11,1,2,38546
1479,20160102,11,0,1,17286


### 집계일자 datetime으로 type 변경 및 집계요일 알아내기

월:0 / 화:1 / 수:2 / 목:3 / 금:4 / 토:5 / 일:6

In [271]:
data_all['집계일자'] = data_all['집계일자'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [272]:
data_all['집계요일'] = data_all['집계일자'].dt.dayofweek

In [273]:
data_all

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량,집계요일
0,2016-01-01,11,0,1,17086,4
1,2016-01-01,11,0,2,36286,4
2,2016-01-01,11,1,1,17633,4
3,2016-01-01,11,1,2,38546,4
1479,2016-01-02,11,0,1,17286,5
1480,2016-01-02,11,0,2,40033,5
1481,2016-01-02,11,1,1,17864,5
1482,2016-01-02,11,1,2,42049,5
2954,2016-01-03,11,0,1,14560,6
2955,2016-01-03,11,0,2,33092,6


### 서울로 향하는 출구 데이터
출구 : 1

#### 월별 총 교통량 분석

1. 해당 월 총합

In [301]:
data_to_seoul_sum = data_all[data_all['입출구구분코드'] == 1]
data_to_seoul_sum['집계일자'] = data_to_seoul_sum['집계일자'].astype(str)
data_to_seoul_sum['집계일자'] = data_to_seoul_sum['집계일자'].apply(lambda x :x[:7])
data_to_seoul_sum

c:\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량,집계요일
2,2016-01,11,1,1,17633,4
3,2016-01,11,1,2,38546,4
1481,2016-01,11,1,1,17864,5
1482,2016-01,11,1,2,42049,5
2956,2016-01,11,1,1,15198,6
2957,2016-01,11,1,2,35299,6
4435,2016-01,11,1,1,20991,2
4436,2016-01,11,1,2,58132,2
5914,2016-01,11,1,1,19725,0
5915,2016-01,11,1,2,55244,0


In [302]:
data_to_seoul_sum = data_to_seoul_sum.groupby(['집계일자','영업소코드'])['총교통량'].sum().reset_index()
data_to_seoul_sum.sort_values(['집계일자', '영업소코드'], ascending=True)

,집계일자,영업소코드,총교통량
0,2016-01,11,2229528
1,2016-01,61,2269633
2,2016-01,62,2903362
3,2016-01,64,2679878
4,2016-01,65,1732435
5,2016-01,66,458937
6,2016-01,68,2812163
7,2016-01,69,150322
8,2016-01,190,1931313
9,2016-01,253,3084225


In [303]:
fig = px.line(data_to_seoul_sum,
       x = '집계일자', y='총교통량', color = '영업소코드')
fig.update_layout(xaxis_rangeslider_visible=True)

많은 순서대로 : 
62(구리남양주) - 68(김포) - 253(서서울) - 64(청계) - 61(성남) - 11(인천) - 190(동서울) - 65(판교) - 66(하남) - 69(대왕판교)

64 61 의 경우 2017년 10월 교통량의 합의 차이가 크게 줄고, 이후 계속해서 비슷한 값을 보인다.

2-1. 평일 - median

In [305]:
data_to_seoul_weekday_med = data_all[data_all['입출구구분코드'] == 1]

In [307]:
# 0부터 4까지. 월요일부터 금요일 데이터만 뽑기
data_to_seoul_weekday_med = data_to_seoul_weekday_med[data_to_seoul_weekday_med['집계요일']<=4]

In [308]:
data_to_seoul_weekday_med['집계일자'] = data_to_seoul_sum['집계일자'].astype(str)
data_to_seoul_sum['집계일자'] = data_to_seoul_sum['집계일자'].apply(lambda x :x[:7])

,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량,집계요일
2,2016-01-01,11,1,1,17633,4
3,2016-01-01,11,1,2,38546,4
4435,2016-01-06,11,1,1,20991,2
4436,2016-01-06,11,1,2,58132,2
5914,2016-01-04,11,1,1,19725,0
5915,2016-01-04,11,1,2,55244,0
7393,2016-01-05,11,1,1,21153,1
7394,2016-01-05,11,1,2,58182,1
8872,2016-01-07,11,1,1,21233,3
8873,2016-01-07,11,1,2,58226,3


2-2. 주말 - median

### 서울에서 나가는 데이터

1. 해당 월 총합

In [296]:
data_from_seoul = data_all[data_all['입출구구분코드'] == 0]
data_from_seoul['집계일자'] = data_from_seoul['집계일자'].astype(str)
data_from_seoul['집계일자'] = data_from_seoul['집계일자'].apply(lambda x :x[:7])
data_from_seoul

c:\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,집계일자,영업소코드,입출구구분코드,TCS하이패스구분코드,총교통량,집계요일
0,2016-01,11,0,1,17086,4
1,2016-01,11,0,2,36286,4
1479,2016-01,11,0,1,17286,5
1480,2016-01,11,0,2,40033,5
2954,2016-01,11,0,1,14560,6
2955,2016-01,11,0,2,33092,6
4433,2016-01,11,0,1,19742,2
4434,2016-01,11,0,2,56047,2
5912,2016-01,11,0,1,18907,0
5913,2016-01,11,0,2,54646,0


In [297]:
data_from_seoul = data_from_seoul.groupby(['집계일자','영업소코드'])['총교통량'].sum().reset_index()
data_from_seoul.sort_values(['집계일자', '영업소코드'], ascending=True)

,집계일자,영업소코드,총교통량
0,2016-01,11,2140585
1,2016-01,61,2380547
2,2016-01,62,2938974
3,2016-01,64,2576103
4,2016-01,65,1575638
5,2016-01,66,499873
6,2016-01,68,2651094
7,2016-01,69,89439
8,2016-01,190,1923905
9,2016-01,253,2837496


In [298]:
fig = px.line(data_to_seoul,
       x = '집계일자', y='총교통량', color = '영업소코드')
fig.update_layout(xaxis_rangeslider_visible=True)

많은 순서대로 <br>
: 62 - 253 - 68 - <br>
2017.10 이전 (64 의왕 - 61 성남) - 11 - 190  - 65 - 66 - 69<br>
2017.10 이후 (61 성남 - 64 성남) - 11 - 190  - 65 - 66 - 69<br>

이유가 뭘까?<br>
청약, 신도시 등으로 인한 인구증가?


In [ ]:
월

---

### 계절별 교통량 분석
어떤 계절에 어느 달에 차가 많을까? 운전하지 말아야짓!!

겨울 : 1월, 2월, 11월, 12월
/봄 : 3월, 4월, 5월
/여름 : 6월, 7월, 8월
/가을 : 9월, 10월 

EDA 해볼 것
* 특정 시간대 (11월 첫째주에 서울에서 안산을 가려고 하는데 월화수목금 중 하루를 정해줘! <br> (반대의 경우도 수행 - 각 톨게이트 맡은 사람이 분석)) --> 이건 의미가 있을까? 교통량으로만 봤을 때 막힐지 이런건 알 수 없잖아요?
* 튀는 데이터를 찾고 이유를 찾아보자
* 봄/여름/가을/겨울에 가장 교통량이 많을 때 --> 이때는 차 운전하지 말자
* 나는 성남에 사는 사람이야! 서울에 회사가 있는데 연차 쓰려구 하는데 언제 쉴까 ? 월요일? 금요일? 
* @ 전체적인 톨게이트 교통량 비교 --> 서울로 향할 때 교통량이 가장 많은 곳 등등,,, 그래프 여러개 한번에 띄워서,,,
* 배추가격 예측 유튜브를 이용해 교통량 예측!
* 날씨, 날짜, 요일에 따른 교통량 예측

* 다혜
각 톨게이트에서 서울로 향해가는 연도별, 월별 시각화/요일별 시각화
* 은지
선형회귀 똑같이 따라했으나 좀 더 확인이 필요
* 단비
계절별 교통량 데이터 분석 및 예측 --> 어느 계절에 어느 달을 추천

교통량 분석 -->

EDA 결과
* 명절에 교통량이 제일 많은 것이 아니다!
* 교통량 1등 톨게이트는 계속 1등임